# NMF - Non-negative matrix factorization

## Init

In [4]:
import numpy as np
import pandas as pd
from scipy import sparse
from functools import partial

import plotly.express as px
from sklearn.preprocessing import OrdinalEncoder

In [17]:
import sys
sys.path.append("../../")
from src.data_preprocessing import TrainTestGenerator
from src.evaluator import Evaluator

In [19]:
data_dir = "../../data/"
data_generator = TrainTestGenerator(data_dir)

## NMF code

In [7]:
def update_H(W, H, V):
    numerator = W.T.dot(V)
    denominator = W.T.dot(W).dot(H) + 1e-10
    H = H*(numerator / denominator)
    return H

def update_W(W, H, V):
    numerator = V.dot(H.T)
    denominator = W.dot(H).dot(H.T) + 1e-10
    W = W*(numerator / denominator)
    return W

In [9]:
def do_nnmf(V, rank=10, iter=100):
    
    # Initialize 
    n, m = V.shape
    
    W = np.abs(np.random.randn(1, n, rank))[0]
    H = np.abs(np.random.randn(1, rank, m))[0]
    
    loss = []
    for i in range(iter):
        H = update_H(W, H, V)
        W = update_W(W, H, V)
        
        loss.append(sum((V - W.dot(H)).flatten()**2))

    return H, W, loss

## Model

In [10]:
# Model wrapper

class NMF_recommender:
    def __init__(self, rank=32, iter=100):
        self.rank = rank
        self.iter = iter
        
    def fit(self, data: pd.DataFrame):
        np.random.seed(1)
        
        # Default rankings when userID is not in training set
        self.default_recommendation = data["artistID"].value_counts().index.tolist()

        data = data.copy()
        
        self.user_encoder = OrdinalEncoder()
        self.artist_encoder = OrdinalEncoder()
        
        data[["userID_ordinal"]] = self.user_encoder.fit_transform(
            data[["userID"]].values
        ).astype(int)
        data[["artistID_ordinal"]] = self.artist_encoder.fit_transform(
            data[["artistID"]].values
        ).astype(int)

        X = np.zeros([data["userID"].nunique(), data["artistID"].nunique()])
        X = np.zeros([data["userID"].nunique(), data["artistID"].nunique()])
        X[data["userID_ordinal"], data["artistID_ordinal"]] = 1        

        H, W, loss = do_nnmf(X, self.rank, self.iter)
        self.H = H
        self.W = W

    def recommend(self, user_id, n):
        try: 
            user_idx = self.user_encoder.transform(np.array([[user_id]]))[0,0]
        except ValueError:
            recommendations = self.default_recommendation
        else:
            user_idx = int(user_idx)

            scores = self.W[user_idx, :].dot(self.H)

            recommendations_idx = np.argsort(scores)
            recommendations_idx = recommendations_idx[:n]

            recommendations = self.artist_encoder.inverse_transform(recommendations_idx.reshape(-1, 1))[:, 0]
        
        return recommendations

## Evaluation

In [20]:
# Evaluator (forward chaining)

evaluator = Evaluator(NMF_recommender, data_generator)
evaluator.evaluate()
evaluator.save_results("../../results/nmf_ranks.csv", "../../results/nmf_times.csv")

In [8]:
# Hit Rate

evaluator.get_hit_rates()

,cases,5,10,25,50,500
2008,4556,0.018437,0.033582,0.054214,0.090430,0.262072
2009,4687,0.024749,0.040751,0.077662,0.111585,0.307019
2010,6133,0.023969,0.052503,0.084298,0.117235,0.300669


In [9]:
# Mean Reciprocal Rank

evaluator.get_mrr()

,cases,mrr
2008,2608,0.025840
2009,3086,0.028101
2010,4306,0.028802


In [10]:
# Times

evaluator.get_times()

,count,mean,std,min,25%,50%,75%,max
task,,,,,,,,
model_fit,3.0,25.838830,18.623034,8.792961,15.900741,23.008522,34.361765,45.715008
model_init,3.0,0.000016,0.000012,0.000003,0.000013,0.000023,0.000023,0.000023
recommend_user,2336.0,0.000671,0.001441,0.000153,0.000228,0.000327,0.000719,0.031549


In [11]:
evaluator.get_fit_per_year_times()

,tag,time
task,,
model_fit,model_fit_2008,8.792961
model_fit,model_fit_2009,23.008522
model_fit,model_fit_2010,45.715008


In [12]:
# Hit Rate
# rank: 20, iter: 150
evaluator = Evaluator(partial(NMF_recommender, rank=20, iter=150), data_generator)
evaluator.evaluate()

evaluator.get_hit_rates()

,cases,5,10,25,50,500
2008,4556,0.018876,0.034021,0.054653,0.091308,0.265584
2009,4687,0.024749,0.040538,0.077448,0.111372,0.306593
2010,6133,0.023969,0.052503,0.084461,0.117398,0.300179
